# YOLOv11 Baseline for License Plate Detection

This notebook implements the training pipeline for the License Plate Detection task using YOLOv11.

**Steps:**
1. **Setup**: Install `ultralytics`.
2. **Data Preparation**: Convert the existing COCO format dataset to YOLO format.
3. **Configuration**: Create the `data.yaml` file.
4. **Training**: Train the YOLOv11 model.
5. **Inference**: Test the model on sample images.

## 1. Setup

In [ ]:
# !pip install ultralytics # Uncomment if you need to install via pip
import ultralytics
ultralytics.checks()

# Monkeypatch to bypass matplotlib font check issue causing RuntimeError
try:
    import ultralytics.utils.checks
    import ultralytics.data.utils
    
    def patch_check_font(font):
        print(f"Skipping font check for {font} to avoid matplotlib error.")
        return

    ultralytics.utils.checks.check_font = patch_check_font
    ultralytics.data.utils.check_font = patch_check_font
    print("Applied workaround for matplotlib font check.")
except Exception as e:
    print(f"Could not apply font check patch: {e}")

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: CUDA is not available. Training will be slow on CPU.")

In [ ]:
import os
import json
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import yaml

## 2. Data Preparation (COCO to YOLO Conversion)

We need to convert the COCO JSON annotations to YOLO `.txt` files.
YOLO format: `class_id x_center y_center width height` (normalized 0-1).

In [ ]:
# Paths
BASE_DIR = "../datasets/IndonesianLiscenePlateDataset/plate_detection_dataset"
ANNOTATIONS_PATH = os.path.join(BASE_DIR, "annotations/annotations.json")
IMAGES_DIR = os.path.join(BASE_DIR, "images")

# Output Directory for YOLO Format
YOLO_DATASET_DIR = "../datasets/yolo_dataset"

# Create directories
for split in ['train', 'val']:
    os.makedirs(os.path.join(YOLO_DATASET_DIR, "images", split), exist_ok=True)
    os.makedirs(os.path.join(YOLO_DATASET_DIR, "labels", split), exist_ok=True)

In [ ]:
def convert_coco_to_yolo(bbox, img_width, img_height):
    """
    Converts COCO bbox [xmin, ymin, w, h] to YOLO bbox [x_center, y_center, w, h] normalized.
    """
    x, y, w, h = bbox
    
    # Center coordinates
    cx = x + w / 2
    cy = y + h / 2
    
    # Normalize
    cx /= img_width
    cy /= img_height
    w /= img_width
    h /= img_height
    
    return [cx, cy, w, h]

In [ ]:
# Load Annotations
with open(ANNOTATIONS_PATH, 'r') as f:
    coco_data = json.load(f)

images = coco_data['images']
annotations = coco_data['annotations']
categories = coco_data['categories']

# Create a map of image_id -> annotations
img_ann_map = {img['id']: [] for img in images}
for ann in annotations:
    img_ann_map[ann['image_id']].append(ann)

# Split Data
train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

print(f"Train images: {len(train_imgs)}")
print(f"Val images: {len(val_imgs)}")

In [ ]:
def process_dataset(image_list, split_name):
    print(f"Processing {split_name} data...")
    
    for img_info in tqdm(image_list):
        img_id = img_info['id']
        file_name = img_info['file_name']
        img_w = float(img_info['width'])
        img_h = float(img_info['height'])
        
        # Source and Destination paths
        src_img_path = os.path.join(IMAGES_DIR, file_name)
        dst_img_path = os.path.join(YOLO_DATASET_DIR, "images", split_name, file_name)
        
        # Copy image
        if os.path.exists(src_img_path):
            shutil.copy(src_img_path, dst_img_path)
        else:
            print(f"Warning: Image not found {src_img_path}")
            continue
            
        # Create Label File
        label_file_name = os.path.splitext(file_name)[0] + ".txt"
        dst_label_path = os.path.join(YOLO_DATASET_DIR, "labels", split_name, label_file_name)
        
        anns = img_ann_map.get(img_id, [])
        
        with open(dst_label_path, 'w') as f:
            for ann in anns:
                # COCO category_id might not start at 0, YOLO expects 0-indexed
                # Assuming 1 category 'license_plate', so class_id = 0
                # If multiple, we need a map. Let's check categories.
                # categories: [{'id': 1, 'name': 'license_plate'}]
                # We'll map id 1 -> 0
                
                class_id = 0 # Since we only have one class
                
                bbox = ann['bbox']
                yolo_bbox = convert_coco_to_yolo(bbox, img_w, img_h)
                
                f.write(f"{class_id} {yolo_bbox[0]:.6f} {yolo_bbox[1]:.6f} {yolo_bbox[2]:.6f} {yolo_bbox[3]:.6f}\n")

# Run conversion
process_dataset(train_imgs, 'train')
process_dataset(val_imgs, 'val')

## 3. Configuration (data.yaml)

In [ ]:
data_yaml = {
    'path': os.path.abspath(YOLO_DATASET_DIR),
    'train': 'images/train',
    'val': 'images/val',
    'nc': 1,
    'names': ['license_plate']
}

yaml_path = os.path.join(YOLO_DATASET_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)
    
print(f"Created data.yaml at {yaml_path}")

## 4. Training YOLOv11

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("../models/yolo11n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(
    data=yaml_path, 
    epochs=50, 
    imgsz=640, 
    batch=16,
    project="../results/detection/yolo11_lpr",
    name="yolo11_run",
    device=0,
    workers=4
)

## 5. Inference / Validation

In [ ]:
# Validate
metrics = model.val()
print(metrics.box.map)  # map50-95

In [ ]:
# Predict on a sample image
import random

val_images_dir = os.path.join(YOLO_DATASET_DIR, "images", "val")
sample_img = random.choice(os.listdir(val_images_dir))
sample_img_path = os.path.join(val_images_dir, sample_img)

results = model(sample_img_path)

# Show result
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im_rgb = cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(im_rgb)
    plt.axis('off')
    plt.show()